In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Chronic_kidney_disease/GSE180393'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Glomerular Transcriptome in the Cprobe Cohort"
!Series_summary	"We used microarrays to analyze the transcriptome of microdissected renal biopsies from patients with kidney disease and living donors. We derived pathway specific scores for Angiopoietin-Tie signaling pathway activation at mRNA level (or transcriptome level) for individual patients and studied the association of pathway activation with disease outcomes."
!Series_overall_design	"Glomerular gene expression data from micro-dissected human kidney biopsy samples  from patients with chronic kidney disease(Lupus, DN, IgA,HT, TN) and healthy living donors. Profiling was performed on Affymetrix ST2.1 microarray platform. "
Sample Characteristics Dictionary:
{0: ['sample group: Living donor', 'sample group: infection-associated GN', 'sample group: FSGS', 'sample group: LN-WHO III', 'sample group: LN-WHO IV', 'sample group: DN', 'sample group: amyloidosis', 'sample group: Membrano-Proliferative 

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Determine if gene expression data is available
is_gene_available = True  # based on the usage of 'microarrays' in the background description

# Determine variable availability and corresponding keys
sample_characteristics = {
    0: ['sample group: Living donor', 'sample group: infection-associated GN', 'sample group: FSGS', 'sample group: LN-WHO III', 'sample group: LN-WHO IV', 'sample group: DN', 'sample group: amyloidosis', 'sample group: Membrano-Proliferative GN', 'sample group: MN', 'sample group: AKI', 'sample group: LN-WHO V', 'sample group: FGGS', "sample group: 2'FSGS", 'sample group: Thin-BMD', 'sample group: Immuncomplex GN', 'sample group: LN-WHO-V', 'sample group: IgAN', 'sample group: LN-WHO IV+V', 'sample group: LN-WHO III+V', 'sample group: LN-WHO-I/II', 'sample group: chronic Glomerulonephritis (GN) with infiltration by CLL', 'sample group: CKD with mod-severe Interstitial fibrosis', 'sample group: Fibrillary GN', 'sample group: Interstitial nephritis', 'sample group: Hypertensive Nephrosclerosis', 'sample group: Unaffected parts of Tumor Nephrectomy'],
    1: ['tissue: Glomeruli from kidney biopsy']
}

# Manually set rows as None since no data for 'Chronic_kidney_disease', 'age', 'gender' available
trait_row = None
age_row = None
gender_row = None

# Define conversion functions if data were available
def convert_trait(value):
    return None  # No Chronic_kidney_disease data available

def convert_age(value):
    return None  # No age data available

def convert_gender(value):
    return None  # No gender data available

# Save cohort info
save_cohort_info('GSE180393', './preprocessed/Chronic_kidney_disease/cohort_info.json', is_gene_available, trait_row is not None)

# Clinical Feature Extraction (skip since `trait_row` is None)
# If trait_row were not None, the code would look like this:
# selected_clinical_data = geo_select_clinical_features(clinical_data, 'Chronic_kidney_disease', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
# csv_path = './preprocessed/Chronic_kidney_disease/trait_data/GSE180393.csv'
# selected_clinical_data.to_csv(csv_path)
# print(preview_df(selected_clinical_data))
